In [8]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [9]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [10]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Add the file to Spark using SparkFiles
spark.sparkContext.addFile(url)
# Read the CSV file into a DataFrame
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)



In [11]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')


In [12]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

avg_4_bd_price = spark.sql("SELECT ROUND(AVG(price),2), YEAR(date) from home_sales where bedrooms == 4 group by YEAR(date) ORDER BY YEAR(date) desc")
avg_4_bd_price.show()

+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           296363.88|      2022|
|           301819.44|      2021|
|           298353.78|      2020|
|            300263.7|      2019|
+--------------------+----------+



In [13]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

price_3b_3ba = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built ORDER BY date_built")
price_3b_3ba.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [14]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

price_3b_3ba_2 = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built")
price_3b_3ba_2.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [15]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

rating = spark.sql("SELECT view, ROUND(AVG(price),2) from home_sales group by view having ROUND(AVG(price),2) >= 350000")
rating.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  81|          1053472.79|
|  76|          1058802.78|
|  91|          1137372.73|
|  93|          1026006.06|
|  52|           733780.26|
|  86|          1070444.25|
|  94|           1033536.2|
|  57|            734340.5|
|  54|           798684.82|
|  96|          1017815.92|
|  92|           970402.55|
|  64|           767036.67|
|  61|           746877.59|
|  88|          1031719.35|
|  72|           780914.67|
|  59|            791453.0|
+----+--------------------+
only showing top 20 rows

--- 1.1588695049285889 seconds ---


In [16]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from home_sales group by view having ROUND(AVG(price),2) >= 350000").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  81|          1053472.79|
|  76|          1058802.78|
|  91|          1137372.73|
|  93|          1026006.06|
|  52|           733780.26|
|  86|          1070444.25|
|  94|           1033536.2|
|  57|            734340.5|
|  54|           798684.82|
|  96|          1017815.92|
|  92|           970402.55|
|  64|           767036.67|
|  61|           746877.59|
|  88|          1031719.35|
|  72|           780914.67|
|  59|            791453.0|
+----+--------------------+
only showing top 20 rows

--- 4.683087587356567 seconds ---


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
output_path = "./output/partitioned_data.parquet"

# Write the DataFrame to Parquet format with partitioning
home_sales_df.write.partitionBy("date_built").parquet(output_path)

In [20]:
# 11. Read the parquet formatted data.
path = "./output/partitioned_data.parquet"
parquet_data = spark.read.parquet(path)
parquet_data

DataFrame[id: string, date: date, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int, date_built: int]

In [21]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("parquet_data")

In [23]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from parquet_data group by view having ROUND(AVG(price),2) >= 350000").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  81|          1053472.79|
|  76|          1058802.78|
|  91|          1137372.73|
|  93|          1026006.06|
|  52|           733780.26|
|  86|          1070444.25|
|  94|           1033536.2|
|  57|            734340.5|
|  54|           798684.82|
|  96|          1017815.92|
|  92|           970402.55|
|  64|           767036.67|
|  61|           746877.59|
|  88|          1031719.35|
|  72|           780914.67|
|  55|           771153.32|
+----+--------------------+
only showing top 20 rows

--- 1.0565271377563477 seconds ---


In [24]:
# 14. Uncache the home_sales temporary table.

spark.catalog.uncacheTable("home_sales")

In [25]:
# 15. Check if the home_sales is no longer cached


spark.catalog.isCached('home_sales')

False